# Profitable app portfolio for apps from Google Play and App Store

* In this project we analyze app profile that are profitable across Google Play and App Store markets. We are working as data analyst for a company that creates free Android and iOS apps, which means most of our profit comes from direct usage of the apps by consumers.  

* Aim of this project is to gain understing about which apps arr most likely to attract users and to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.


In [3]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [4]:
from csv import reader
as_data = open('AppleStore.csv')
app_store = list(reader(as_data))
gp_data = open('googleplaystore.csv')
google_play = list(reader(gp_data))

In [5]:
explore_data(app_store,0,15, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


['429047995', 'Pinterest', '74778624', 'USD', '0.0', '1061

In [6]:
explore_data(google_play,0,3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


In [7]:
app_store_header = app_store[0]
app_store = app_store[1:]
google_play_header = google_play[0]
google_play = google_play[1:]

print(app_store_header)
print(google_play_header)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [8]:
# correct incorrect row
#del google_play[10472]
print(google_play[10472])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


# Cleaning Google Play dataset

 According to discussion on kaggle there is duplicate data in goole play data set, i.e. there are multiple entries for the same app:

In [9]:
for app in google_play:
    if (app[0] == 'Coloring book moana'):
        print(app)

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']
['Coloring book moana', 'FAMILY', '3.9', '974', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


In following code we will count total of dublicate and unique app entries in the data set:

In [10]:
unique_apps = []
duplicate_apps = []

for app in google_play:
    name = app[0]
    if name not in unique_apps:
        unique_apps.append(name)
    else:
        duplicate_apps.append(name)
        
print('Number of uniue entries: ', len(unique_apps))
print('Number of duplicated entries: ', len(duplicate_apps))

Number of uniue entries:  9659
Number of duplicated entries:  1181


In the next step we will proceed to remove duplicates based on the number of riviews. We could use different criterion, but as you can see in the example above, there might not be any differences. Thus, the entry with more reviews will be selected, because it indicates that this entry was the latest one. After duplicate removal we should get **9659** numer of rows (see calculation above).

In [11]:
reviews_max = {}

for app in google_play:
    name = app[0]
    n_reviews = float(app[3])
    
    if(name not in reviews_max) or (reviews_max[name] < n_reviews):
        reviews_max[name] = n_reviews
    
    
print(reviews_max.get('Coloring book moana'))

974.0


In [12]:
android_clean = []
already_added = []

for app in google_play:
    name = app[0]
    n_reviews = float(app[3])
    
    if(n_reviews == reviews_max[name] and name not in already_added):
        android_clean.append(app)
        already_added.append(name)
        
explore_data(android_clean, 1, 3, True)

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


# Removing non english apps

In [13]:
def is_english(app_name):
    non_ascii  = 0;
    
    for char in app_name:
        if ord(char) > 127:
            non_ascii += 1
            
    if non_ascii > 3:
        return False
    return True

print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))

False
True


In [14]:
def filter_english_apps(list_of_apps, app_name_column_index):
    clean_list = []
    for app in list_of_apps:
        if(is_english(app[app_name_column_index])):
            clean_list.append(app)
    return clean_list

In [15]:
google_play_eng = filter_english_apps(android_clean ,0)
explore_data(google_play_eng,0,3,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


In [16]:
app_store_eng = filter_english_apps(app_store, 1)
explore_data(app_store_eng,0,3,True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 6183
Number of columns: 16


# Isolating free apps

since our company develops only free apps, we only care abut data concerning free application. Both data sets have column 'price', 5th in App Store data set and 8th in Google Play. 

In [17]:
print(app_store_header.index('price'))
print(google_play_header.index('Price'))

4
7


In [18]:
def filter_free_apps(list_of_apps, price_column_index):
    result = []
    for app in list_of_apps:
        if app[price_column_index] == '0' or app[price_column_index] == '0.0':
            result.append(app)
    return result

ios_final = filter_free_apps(app_store_eng, app_store_header.index('price'))
android_final = filter_free_apps(google_play_eng, google_play_header.index('Price'))

In [19]:
print(len(ios_final))
print(len(android_final))

3222
8864


# Identifying optimal app profile

As end goal for apps designed by our company to be available both to Android and iOS users, we need to get a sense of which application do well in both market spaces. To do that we will find common ground between popular application in both data sets, starting with genres. To do that we will generate frequency tables for both data sets and see which type of app we should focus are development efforts on.

In [24]:
print(google_play_header.index('Category'))
print(google_play_header.index('Genres'))
print(app_store_header.index('prime_genre'))

1
9
11


In [28]:
def freq_table(dataset, index):
    frequency_table = {}
    for app in dataset:
        genre = app[index]
        if genre in frequency_table:
            frequency_table[genre] += 1
        else:
            frequency_table[genre] = 1
    return frequency_table

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [30]:
display_table(android_final, google_play_header.index('Category'))

FAMILY : 1676
GAME : 862
TOOLS : 750
BUSINESS : 407
LIFESTYLE : 346
PRODUCTIVITY : 345
FINANCE : 328
MEDICAL : 313
SPORTS : 301
PERSONALIZATION : 294
COMMUNICATION : 287
HEALTH_AND_FITNESS : 273
PHOTOGRAPHY : 261
NEWS_AND_MAGAZINES : 248
SOCIAL : 236
TRAVEL_AND_LOCAL : 207
SHOPPING : 199
BOOKS_AND_REFERENCE : 190
DATING : 165
VIDEO_PLAYERS : 159
MAPS_AND_NAVIGATION : 124
FOOD_AND_DRINK : 110
EDUCATION : 103
ENTERTAINMENT : 85
LIBRARIES_AND_DEMO : 83
AUTO_AND_VEHICLES : 82
HOUSE_AND_HOME : 73
WEATHER : 71
EVENTS : 63
PARENTING : 58
ART_AND_DESIGN : 57
COMICS : 55
BEAUTY : 53


In [31]:
display_table(android_final, google_play_header.index('Genres'))

Tools : 749
Entertainment : 538
Education : 474
Business : 407
Productivity : 345
Lifestyle : 345
Finance : 328
Medical : 313
Sports : 307
Personalization : 294
Communication : 287
Action : 275
Health & Fitness : 273
Photography : 261
News & Magazines : 248
Social : 236
Travel & Local : 206
Shopping : 199
Books & Reference : 190
Simulation : 181
Dating : 165
Arcade : 164
Video Players & Editors : 157
Casual : 156
Maps & Navigation : 124
Food & Drink : 110
Puzzle : 100
Racing : 88
Role Playing : 83
Libraries & Demo : 83
Auto & Vehicles : 82
Strategy : 81
House & Home : 73
Weather : 71
Events : 63
Adventure : 60
Comics : 54
Beauty : 53
Art & Design : 53
Parenting : 44
Card : 40
Casino : 38
Trivia : 37
Educational;Education : 35
Board : 34
Educational : 33
Education;Education : 30
Word : 23
Casual;Pretend Play : 21
Music : 18
Racing;Action & Adventure : 15
Puzzle;Brain Games : 15
Entertainment;Music & Video : 15
Casual;Brain Games : 12
Casual;Action & Adventure : 12
Arcade;Action & Advent

In [33]:
display_table(ios_final, app_store_header.index('prime_genre'))

Games : 1874
Entertainment : 254
Photo & Video : 160
Education : 118
Social Networking : 106
Shopping : 84
Utilities : 81
Sports : 69
Music : 66
Health & Fitness : 65
Productivity : 56
Lifestyle : 51
News : 43
Travel : 40
Finance : 36
Weather : 28
Food & Drink : 26
Reference : 18
Business : 17
Book : 14
Navigation : 6
Medical : 6
Catalogs : 4


# Most Popular Apps by Genre on the App Store

As we can see both markets have a lean towards entertainment apps, but that does not indicate that these application ar downloaded and used the most. To get that information weļl collect installtion / rating number for each genre, which will represent popularity of applications of specific genre among users.

In [42]:
prime_gen_index = app_store_header.index('prime_genre')
prime_genre_freq = freq_table(ios_final, prime_gen_index)

for genre in prime_genre_freq:
    total = 0
    len_genre = 0
    for app in ios_final:
        genre_app = app[prime_gen_index]
        if genre_app == genre:
            total += float(app[app_store_header.index('rating_count_tot')])
            len_genre += 1
    avg_total = total/len_genre
    print(genre, ": ", avg_total)

Social Networking :  71548.34905660378
Education :  7003.983050847458
Catalogs :  4004.0
Lifestyle :  16485.764705882353
Finance :  31467.944444444445
Photo & Video :  28441.54375
News :  21248.023255813954
Navigation :  86090.33333333333
Health & Fitness :  23298.015384615384
Utilities :  18684.456790123455
Entertainment :  14029.830708661417
Book :  39758.5
Sports :  23008.898550724636
Shopping :  26919.690476190477
Travel :  28243.8
Music :  57326.530303030304
Productivity :  21028.410714285714
Business :  7491.117647058823
Medical :  612.0
Reference :  74942.11111111111
Games :  22788.6696905016
Weather :  52279.892857142855
Food & Drink :  33333.92307692308


In [43]:
for app in ios_final:
    if app[prime_gen_index] == 'Social Networking':
        print(app[1], ": ", app[5])

Facebook :  2974676
Pinterest :  1061624
Skype for iPhone :  373519
Messenger :  351466
Tumblr :  334293
WhatsApp Messenger :  287589
Kik :  260965
ooVoo – Free Video Call, Text and Voice :  177501
TextNow - Unlimited Text + Calls :  164963
Viber Messenger – Text & Call :  164249
Followers - Social Analytics For Instagram :  112778
MeetMe - Chat and Meet New People :  97072
We Heart It - Fashion, wallpapers, quotes, tattoos :  90414
InsTrack for Instagram - Analytics Plus More :  85535
Tango - Free Video Call, Voice and Chat :  75412
LinkedIn :  71856
Match™ - #1 Dating App. :  60659
Skype for iPad :  60163
POF - Best Dating App for Conversations :  52642
Timehop :  49510
Find My Family, Friends & iPhone - Life360 Locator :  43877
Whisper - Share, Express, Meet :  39819
Hangouts :  36404
LINE PLAY - Your Avatar World :  34677
WeChat :  34584
Badoo - Meet New People, Chat, Socialize. :  34428
Followers + for Instagram - Follower Analytics :  28633
GroupMe :  28260
Marco Polo Video Walki

In [44]:
for app in ios_final:
    if app[prime_gen_index] == 'Navigation':
        print(app[1], ": ", app[5])

Waze - GPS Navigation, Maps & Real-time Traffic :  345046
Google Maps - Navigation & Transit :  154911
Geocaching® :  12811
CoPilot GPS – Car Navigation & Offline Maps :  3582
ImmobilienScout24: Real Estate Search in Germany :  187
Railway Route Search :  5


In [41]:
for app in ios_final:
    if app[prime_gen_index] == 'Reference':
        print(app[1], ": ", app[5])

Bible :  985920
Dictionary.com Dictionary & Thesaurus :  200047
Dictionary.com Dictionary & Thesaurus for iPad :  54175
Google Translate :  26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran :  18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition :  17588
Merriam-Webster Dictionary :  16849
Night Sky :  12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) :  8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools :  4693
GUNS MODS for Minecraft PC Edition - Mods Tools :  1497
Guides for Pokémon GO - Pokemon GO News and Cheats :  826
WWDC :  762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free :  718
VPN Express :  14
Real Bike Traffic Rider Virtual Reality Glasses :  8
教えて!goo :  0
Jishokun-Japanese English Dictionary & Translator :  0


**Conclusion: ** Judging by accuired data we can see, that despite Soial networking and Navigation apps are at the top, the market is dominated by well known names, whereas Refrence genre looks more promising. Any guide or tech book can be converted into an app fromat with added features, which would fit this category.

# Most Popular Apps by Genre on the Play Store

In [61]:
gen_index = google_play_header.index('Category')
genre_freq = freq_table(android_final, gen_index)

for genre in genre_freq:
    total = 0
    len_genre = 0
    for app in android_final:
        genre_app = app[gen_index]
        if genre_app == genre:
            str_total = app[google_play_header.index('Installs')].replace('+','') 
            str_total = str_total.replace(',','')
            total += float(str_total)
            len_genre += 1
    avg_total = total/len_genre
#     if(20000000 > avg_total > 10000000):
    print(genre, ": ", avg_total)

NEWS_AND_MAGAZINES :  9549178.467741935
MAPS_AND_NAVIGATION :  4056941.7741935486
FINANCE :  1387692.475609756
MEDICAL :  120550.61980830671
HEALTH_AND_FITNESS :  4188821.9853479853
PRODUCTIVITY :  16787331.344927534
HOUSE_AND_HOME :  1331540.5616438356
TRAVEL_AND_LOCAL :  13984077.710144928
LIBRARIES_AND_DEMO :  638503.734939759
EDUCATION :  1833495.145631068
SHOPPING :  7036877.311557789
FAMILY :  3695641.8198090694
COMICS :  817657.2727272727
TOOLS :  10801391.298666667
GAME :  15588015.603248259
AUTO_AND_VEHICLES :  647317.8170731707
PARENTING :  542603.6206896552
WEATHER :  5074486.197183099
COMMUNICATION :  38456119.167247385
PERSONALIZATION :  5201482.6122448975
BEAUTY :  513151.88679245283
SPORTS :  3638640.1428571427
BOOKS_AND_REFERENCE :  8767811.894736841
BUSINESS :  1712290.1474201474
ENTERTAINMENT :  11640705.88235294
DATING :  854028.8303030303
SOCIAL :  23253652.127118643
FOOD_AND_DRINK :  1924897.7363636363
LIFESTYLE :  1437816.2687861272
ART_AND_DESIGN :  1986335.08771

In [56]:
for app in android_final:
    if app[gen_index] == 'COMMUNICATION':
        print(app[0], ": ", app[5])

WhatsApp Messenger :  1,000,000,000+
Messenger for SMS :  10,000,000+
My Tele2 :  5,000,000+
imo beta free calls and text :  100,000,000+
Contacts :  50,000,000+
Call Free – Free Call :  5,000,000+
Web Browser & Explorer :  5,000,000+
Browser 4G :  10,000,000+
MegaFon Dashboard :  10,000,000+
ZenUI Dialer & Contacts :  10,000,000+
Cricket Visual Voicemail :  10,000,000+
TracFone My Account :  1,000,000+
Xperia Link™ :  10,000,000+
TouchPal Keyboard - Fun Emoji & Android Keyboard :  10,000,000+
Skype Lite - Free Video Call & Chat :  5,000,000+
My magenta :  1,000,000+
Android Messages :  100,000,000+
Google Duo - High Quality Video Calls :  500,000,000+
Seznam.cz :  1,000,000+
Antillean Gold Telegram (original version) :  100,000+
AT&T Visual Voicemail :  10,000,000+
GMX Mail :  10,000,000+
Omlet Chat :  10,000,000+
My Vodacom SA :  5,000,000+
Microsoft Edge :  5,000,000+
Messenger – Text and Video Chat for Free :  1,000,000,000+
imo free video calls and chat :  500,000,000+
Calls & Tex

In [67]:
for app in android_final:
    if app[gen_index] == 'BOOKS_AND_REFERENCE' and (app[5] == '5,000,000+' or app[5] == '1,000,000+'):
        print(app[0], ": ", app[5])

Book store :  1,000,000+
Free Books - Spirit Fanfiction and Stories :  1,000,000+
AlReader -any text book reader :  5,000,000+
FamilySearch Tree :  1,000,000+
Cloud of Books :  1,000,000+
ReadEra – free ebook reader :  1,000,000+
Ebook Reader :  5,000,000+
Read books online :  5,000,000+
eBoox: book reader fb2 epub zip :  1,000,000+
All Maths Formulas :  1,000,000+
Ancestry :  5,000,000+
English-Myanmar Dictionary :  1,000,000+
Golden Dictionary (EN-AR) :  1,000,000+
All Language Translator Free :  1,000,000+
Dictionary - WordWeb :  5,000,000+
50000 Free eBooks & Free AudioBooks :  5,000,000+
Al Quran Al karim :  1,000,000+
Al Quran : EAlim - Translations & MP3 Offline :  5,000,000+
Koran Read &MP3 30 Juz Offline :  1,000,000+
Hafizi Quran 15 lines per page :  1,000,000+
Satellite AR :  1,000,000+
Oxford A-Z of English Usage :  1,000,000+
Bible KJV :  5,000,000+
Brilliant Quotes: Life, Love, Family & Motivation :  1,000,000+
Stats Royale for Clash Royale :  1,000,000+
wikiHow: how to d

**Conclusion: ** Same thing is prevalent here as in app store : social, communication and navigation application, as well as video players are on top, but the statistics are scewed by tech giants like google and michrosoft. If we take a look at the book and Reference genre on the other hand the distribution is more or less even across all kinds of applications. It seems book readers and books converted  